# Twitter Tweets Analysis as positive or negative

Getting necessary libraries

In [1]:
import numpy as np
import pandas as pd
import nltk 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [2]:
testing_data = pd.read_csv('0000000000002747_test_twitter_x_test (1).csv')
training_data= pd.read_csv('0000000000002747_training_twitter_x_y_train (1).csv')

In [3]:
print(training_data.columns)
print(testing_data.columns)

Index(['tweet_id', 'airline_sentiment', 'airline', 'airline_sentiment_gold',
       'name', 'negativereason_gold', 'retweet_count', 'text', 'tweet_coord',
       'tweet_created', 'tweet_location', 'user_timezone'],
      dtype='object')
Index(['tweet_id', 'airline', 'airline_sentiment_gold', 'name',
       'negativereason_gold', 'retweet_count', 'text', 'tweet_coord',
       'tweet_created', 'tweet_location', 'user_timezone'],
      dtype='object')


In [4]:
training_result = training_data['airline_sentiment']

In [5]:
training_data

tweet_id airline_sentiment     airline  \
0      567900433542488064          negative   Southwest   
1      569989168903819264          positive   Southwest   
2      568089179520954368          positive      United   
3      568928195581513728          negative   Southwest   
4      568594180014014464          negative      United   
5      569677636613439488          negative      United   
6      569658903044218880          negative       Delta   
7      568542766860541952          positive       Delta   
8      570116209263427584          negative      United   
9      568870144891600896          positive   Southwest   
10     567793635892174848          negative   Southwest   
11     569879497891303424           neutral       Delta   
12     568517827482685440          positive  US Airways   
13     568212893012860928          negative   Southwest   
14     569270187725160449          negative  US Airways   
15     567816651900583936          negative  US Airways   
16     569690131235340288          negative    American   
17     569932678688055296          negative       Delta   
18     569939494465462272          negative       Delta   
19     569958224968982528          negative    American   
20     568361346464497664          negative  US Airways   
21     568126682336534529           neutral   Southwest   
22     568883826333777920          negative  US Airways   
23     570055700790931456          negative      United   
24     568804026789138433          positive      United   
25     567925054853496832           neutral      United   
26     568434301496381441          negative       Delta   
27     570083397873373184          negative      United   
28     568585356586586113          positive   Southwest   
29     570022265343123456          positive    American   
...                   ...               ...         ...   
10950  569125697530101760          positive       Delta   
10951  569682747712770048          negative   Southwest   
10952  569372333267394560          negative   Southwest   
10953  569333972452265985          negative      United   
10954  568426579874996224          negative   Southwest   
10955  570018254271463424          negative       Delta   
10956  568118177537744897           neutral   Southwest   
10957  568522861205266432          negative  US Airways   
10958  570290993339244547          negative      United   
10959  569624009354170369          negative  US Airways   
10960  568841631585021952          negative      United   
10961  569262534232039427          negative       Delta   
10962  569308255966666753           neutral      United   
10963  569369008580874240          negative  US Airways   
10964  569008708174262272          negative      United   
10965  570164810144481280           neutral      United   
10966  568417151176060929          negative      United   
10967  567617081336950784          negative   Southwest   
10968  569664787946164224          negative    American   
10969  568531020267458561          positive   Southwest   
10970  570283248301043712          negative   Southwest   
10971  569087442092826624           neutral       Delta   
10972  570051488875020288          negative  US Airways   
10973  569676702030262272          negative    American   
10974  569676404893163520          positive   Southwest   
10975  569934458364813313           neutral    American   
10976  568564006329434113          positive      United   
10977  569643648910028801          negative  US Airways   
10978  568864981917110272          negative  US Airways   
10979  568929299350179840          negative      United   

      airline_sentiment_gold             name negativereason_gold  \
0                        NaN    ColeyGirouard                 NaN   
1                        NaN    WalterFaddoul                 NaN   
2                        NaN        LocalKyle                 NaN   
3                        NaN      amccarthy19                 NaN   
4            

Removing unwanted features

In [6]:
def updated_data(data):
    del data['tweet_id']
    del data['name']
    del data['tweet_created']
    del data['tweet_location']
    del data['user_timezone']
    del data['tweet_coord']
    del data['airline_sentiment']
    del data['airline']
    del data['negativereason_gold']
    del data['retweet_count']
    del data['airline_sentiment_gold']
    return data

def updated_data1(data):
    del data['tweet_id']
    del data['name']
    del data['tweet_created']
    del data['tweet_location']
    del data['user_timezone']
    del data['tweet_coord']
    del data['airline']
    del data['negativereason_gold']
    del data['retweet_count']
    del data['airline_sentiment_gold']
    return data

In [7]:
training_data = updated_data(training_data)
testing_data = updated_data1(testing_data)

In [8]:
testing_data

text
0     @AmericanAir In car gng to DFW. Pulled over 1h...
1     @AmericanAir after all, the plane didn’t land ...
2     @SouthwestAir can't believe how many paying cu...
3     @USAirways I can legitimately say that I would...
4     @AmericanAir still no response from AA. great ...
5     @united we have developers flying down tmrw mo...
6                     @USAirways hello??? Anyone there?
7     @USAirways @husainhaqqani Mr. Husain u shld pr...
8     @USAirways not likely, flightaware says plane ...
9     @AmericanAir they don't even give an option to...
10    @united your announcement for pre boarding onl...
11    @USAirways it is really embarrassing when aski...
12    @SouthwestAir I will not have my passport in t...
13    @AmericanAir this delayed bag was for my frien...
14    @SouthwestAir Didn't see travel had to be comp...
15    @USAirways awesome... Doors close in 2 minutes...
16    @united I flew United last month and the exper...
17    @JetBlue Accepting #Apple Pay - Mobile Enterpr...
18    @united $55 cab ride to dfw from love got me m...
19    @JetBlue ill call in the morning. too upset ri...
20    @SouthwestAir @AARP #tfw1 Appreciate the tweet...
21    @united what's the status of flight 1008 Bogot...
22    @AmericanAir literally just stopped allowing p...
23    @AmericanAir I really want to get home. Tonigh...
24    @USAirways I've been on hold at the reservatio...
25    @JetBlue Denver-Boston 2/10 flight 994  and Bo...
26    @AmericanAir nothing to do with Mother Nature,...
27    @USAirways you would rather a while plane get ...
28    @JetBlue hi guys, do you have a general enquir...
29    @united we finally just arrive to Bogota, good...
...                                                 ...
3630                         @USAirways nope not yet...
3631  @SouthwestAir So far so good! http://t.co/16c9...
3632  @SouthwestAir @CBSsoxfan you get to hold? call...
3633  @JetBlue Really!? That's good to hear! Thanks ...
3634  @JetBlue trying to charge us $550 dollars cash...
3635  @USAirways can u tell me if 5238 from clt to J...
3636  @USAirways Your customer service is  non-exist...
3637  @SouthwestAir Whoa. Thanks and that's what I w...
3638  @United THANK U! Secured room for the night Th...
3639  @AmericanAir I'm not sure what happened to my ...
3640  @AmericanAir pretty bad at #DFW no updates two...
3641  @USAirways Just contaced EYEWITNESS NEWS about...
3642  @SouthwestAir thx. Make it right. Help Meagan ...
3643  @USAirways I didn't Cancelled Flight my flight...
3644  @united your "Airserv" contractors aren't wort...
3645  @SouthwestAir @taylormdowns We share that valu...
3646  @USAirways on hold now over 2 hrs on one phone...
3647  @AmericanAir the flight I'm trying to change i...
3648  @JetBlue the fact that #ChrisHasMadeUsBLUSH is...
3649           @united Did somebody say flight upgrade?
3650      @SouthwestAir how do I get my companion pass?
3651  @USAirways tells me to talk to @AmericanAir ab...
3652          @JetBlue you want me to talk to the wall?
3653  @USAirways Now, the JFK Baggage Office has act...
3654  @AmericanAir Least you could do would be to re...
3655  @USAirways Been stuck for 40+ minutes due to l...
3656  @USAirways 4 hours... 4 hours... FOUR HOURS.  ...
3657  Nice RT @VirginAmerica: The man of steel might...
3658  @AmericanAir Aww Thanks AA..DFW was on GMA up ...
3659  @united the lounge tells us they have no pillo...

[3660 rows x 1 columns]

Getting Stop words and punctuations

In [9]:
lemmatizer = WordNetLemmatizer()
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)

In [10]:
def getSimpleTag(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [11]:
def cleanWords(words):
    words = word_tokenize(words)
    output_words = []
    for w in words:
        if w.lower() not in stops:
            #w1 = lemmatizer.lemmatize(w.lower())
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = getSimpleTag(pos[0][1]))
            output_words.append(clean_word)      
    return output_words

In [12]:
i = 0
for doc in training_data['text']:
    print(doc)
    if i == 2:
        break
    i+=1

@SouthwestAir I am scheduled for the morning, 2 days after the fact, yes..not sure why my evening flight was the only one Cancelled Flightled
@SouthwestAir seeing your workers time in and time out going above and beyond is why I love flying with you guys. Thank you!
@united Flew ORD to Miami and back and  had great crew, service on both legs. THANKS


In [13]:
words_arr = [cleanWords(doc) for doc in training_data['text']]

Using pre computed saved data

In [29]:
import pickle
pickle_out = open('words_arr.txt', 'wb')
pickle.dump(words_arr, pickle_out)
pickle_out.close()

In [14]:
import pickle
pickle_in = open('words_arr.txt', 'rb')
words_arr = pickle.load(pickle_in)

In [15]:
words_arr

[['SouthwestAir',
  'schedule',
  'morning',
  '2',
  'days',
  'fact',
  'yes..not',
  'sure',
  'even',
  'flight',
  'one',
  'Cancelled',
  'Flightled'],
 ['SouthwestAir',
  'see',
  'workers',
  'time',
  'time',
  'go',
  'beyond',
  'love',
  'fly',
  'guys',
  'Thank'],
 ['united',
  'Flew',
  'ORD',
  'Miami',
  'back',
  'great',
  'crew',
  'service',
  'legs',
  'THANKS'],
 ['SouthwestAir', 'dultch97', "'s", 'horse', 'radish', '😤🐴'],
 ['united',
  'flight',
  'ORD',
  'delayed',
  'Air',
  'Force',
  'One',
  'last',
  'flight',
  'SBN',
  '8:20',
  '5',
  'mins',
  'land'],
 ['united',
  'load',
  'u',
  'fly',
  'sardine',
  'knew',
  'pilots',
  '2',
  'hours',
  'Late',
  'Flight',
  'incompetent',
  'beyond',
  'belief'],
 ['JetBlue',
  'stock',
  'response',
  'Delays',
  'frustrate',
  'poor',
  'cust',
  'serv',
  'amp',
  'told',
  '3',
  'ppl',
  'wait',
  'amp',
  "'d",
  'come',
  'back'],
 ['JetBlue',
  "'d",
  'nice',
  'Hoping',
  'rack',
  'enough',
  'miles

In [15]:
stemmed_words = []
i = 0
for doc in words_arr:
    stemmed_words.append([])
    for w in doc:
        stemmed_words[i].append(ps.stem(w))
    i += 1

In [17]:
words_arr[0]

['SouthwestAir',
 'schedule',
 'morning',
 '2',
 'days',
 'fact',
 'yes..not',
 'sure',
 'even',
 'flight',
 'one',
 'Cancelled',
 'Flightled']

In [18]:
all_words = []
for doc in words_arr:
    all_words += doc

In [19]:
freq = nltk.FreqDist(all_words)
common = freq.most_common(1000)
features = [i[0] for i in common]

In [20]:
count_vec = CountVectorizer(max_features = 3000, ngram_range = (1, 2))
train_data = count_vec.fit_transform(all_words)

In [21]:
count_vec.get_feature_names()

['00',
 '000',
 '00pm',
 '02',
 '03',
 '05',
 '10',
 '10 000',
 '10 24',
 '10 30',
 '100',
 '1000',
 '10pm',
 '11',
 '11 30',
 '11th',
 '12',
 '1230',
 '13',
 '130',
 '14',
 '140',
 '15',
 '150',
 '152',
 '15th',
 '16',
 '17',
 '18',
 '180',
 '1800',
 '19',
 '1hr',
 '1k',
 '1st',
 '20',
 '200',
 '2014',
 '2015',
 '20min',
 '21',
 '22',
 '23',
 '24',
 '24h',
 '24hrs',
 '24th',
 '25',
 '250',
 '26',
 '27',
 '28',
 '28th',
 '29',
 '2b',
 '2d',
 '2day',
 '2days',
 '2hr',
 '2hrs',
 '2nd',
 '2pm',
 '2x',
 '30',
 '30 60',
 '300',
 '3056',
 '30a',
 '30am',
 '30k',
 '30min',
 '30pm',
 '31',
 '3130',
 '32',
 '35',
 '350',
 '36',
 '386',
 '39',
 '3am',
 '3d',
 '3hrs',
 '3pm',
 '3rd',
 '3x',
 '40',
 '400',
 '404',
 '40pm',
 '40th',
 '413',
 '42',
 '428',
 '428 4322',
 '4322',
 '433',
 '433 7300',
 '435',
 '44',
 '440',
 '445',
 '45',
 '4567',
 '45min',
 '45mins',
 '47',
 '475',
 '48',
 '4am',
 '4ojrsdwpkk',
 '4pm',
 '4th',
 '4x',
 '50',
 '500',
 '501',
 '50am',
 '50k',
 '50pm',
 '51',
 '5350',
 '5

In [22]:
train_data = count_vec.transform(training_data['text'])

In [23]:
test_data = count_vec.transform(testing_data['text'])

In [24]:
train_data

<10980x3000 sparse matrix of type '<class 'numpy.int64'>'
	with 132295 stored elements in Compressed Sparse Row format>

In [25]:
test_data

<3660x3000 sparse matrix of type '<class 'numpy.int64'>'
	with 43572 stored elements in Compressed Sparse Row format>

In [26]:
training_result.shape, train_data.shape

((10980,), (10980, 3000))

Applying different Classifiers

Training on SVM Classifier

In [81]:
from sklearn.svm import SVC

In [82]:
svc = SVC()
svc.fit(train_data, training_result)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

getting score from svm classifier

In [83]:
svc.score(train_data, training_result)

0.6252276867030966

In [84]:
y_pred_svc = svc.predict(test_data)

In [85]:
set(y_pred_svc)

{'negative', 'positive'}

In [87]:
np.savetxt('y_pred_svc.csv', y_pred_svc, fmt = "%s")

Training on Random Forest Classifier

In [103]:
rnd_clf = RandomForestClassifier(criterion = 'gini', n_estimators = 25, n_jobs = 2, verbose = 7)
rnd_clf.fit(train_data, training_result)

building tree 1 of 25building tree 2 of 25

building tree 3 of 25
building tree 4 of 25
building tree 5 of 25
building tree 6 of 25


[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.5s


building tree 7 of 25
building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=2)]: Done  25 out of  25 | elapsed:    3.5s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=2,
            oob_score=False, random_state=None, verbose=7,
            warm_start=False)

getting score from random forest classifier

In [104]:
print(rnd_clf.score(train_data, training_result))

[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.0s


0.9936247723132969


[Parallel(n_jobs=2)]: Done  25 out of  25 | elapsed:    0.0s finished


In [105]:
y_pred_rnd = rnd_clf.predict(test_data)
print(set(y_pred_rnd))

[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  25 out of  25 | elapsed:    0.0s finished


{'neutral', 'positive', 'negative'}


In [107]:
np.savetxt('y_pred_rnd.csv', y_pred_rnd, fmt = "%s")